In [ ]:
# libraryをimportする
import os
import re
import zipfile
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [39]:
# 江戸川乱歩の作者ページを設定する
edokawa_url = "https://www.aozora.gr.jp/index_pages/person1779.html"


# 青空文庫のページをUTF-8のencodingで取得する
response = requests.get(edokawa_url)
response.encoding = 'utf-8'

# BeautifulSoupを用いで、HTMLを解析する
soup = BeautifulSoup(response.text, 'html.parser')

# 作品リストを取得する
books_list = soup.select('ol li a')  

# 情報を抽出する
books = []
for book in books_list:
    relative_link = book['href'] 

    # 「person」が含まるlinkを無視する
    if "person" in relative_link:
        continue

    full_link = f"https://www.aozora.gr.jp{relative_link[2:]}"  # full linkに転換する
    book_name = book.text.strip()  # 作品名を収集する
    book_id = relative_link.split('/')[-1].replace('card', '').replace('.html', '')  # 作品IDを収集する

    # 作品の情報を基づいてdictionaryを作る
    books.append({book_id: [book_name, full_link]})

# dictionaryをprintする
for book in books:
    print(book)


{'57105': ['赤いカブトムシ', 'https://www.aozora.gr.jp/cards/001779/card57105.html']}
{'57181': ['赤い部屋', 'https://www.aozora.gr.jp/cards/001779/card57181.html']}
{'57240': ['悪魔の紋章', 'https://www.aozora.gr.jp/cards/001779/card57240.html']}
{'57515': ['悪霊', 'https://www.aozora.gr.jp/cards/001779/card57515.html']}
{'58699': ['悪霊物語', 'https://www.aozora.gr.jp/cards/001779/card58699.html']}
{'59399': ['「悪霊物語」自作解説', 'https://www.aozora.gr.jp/cards/001779/card59399.html']}
{'58333': ['暗黒星', 'https://www.aozora.gr.jp/cards/001779/card58333.html']}
{'57531': ['偉大なる夢', 'https://www.aozora.gr.jp/cards/001779/card57531.html']}
{'57182': ['一枚の切符', 'https://www.aozora.gr.jp/cards/001779/card57182.html']}
{'58053': ['一寸法師', 'https://www.aozora.gr.jp/cards/001779/card58053.html']}
{'57503': ['陰獣', 'https://www.aozora.gr.jp/cards/001779/card57503.html']}
{'56674': ['宇宙怪人', 'https://www.aozora.gr.jp/cards/001779/card56674.html']}
{'58590': ['江川蘭子', 'https://www.aozora.gr.jp/cards/001779/card58590.html']}
{'572

In [ ]:
def clean_and_extract_publication_year(text):
    # 出版した時間を抽出する
    cleaned_text = re.sub(r"（[^）]*）", "", text)
    pattern = r"(\d{4})年"
    match = re.search(pattern, cleaned_text)
    if match:
        return match.group(1)
    return None

def extract_download_links(soup, base_url):
    # Webからすべての .zip download linkを抽出する
    download_links = []
    download_table = soup.find('table', class_='download')
    if download_table:
        download_rows = download_table.find_all('tr', bgcolor='white')
        for row in download_rows:
            link_tag = row.find('a', href=True)
            if link_tag and link_tag['href'].endswith('.zip'):
                full_url = urljoin(base_url, link_tag['href'])
                download_links.append(full_url)
    return download_links

def download_file(url, save_folder):
    # URLに従ってfileをdownloadし、指定されたfolderに保存する
    response = requests.get(url)
    response.raise_for_status()
    filename = url.split('/')[-1]
    save_path = os.path.join(save_folder, filename)
    os.makedirs(save_folder, exist_ok=True)
    with open(save_path, 'wb') as file:
        file.write(response.content)
    return save_path

def extract_zip(zip_path, extract_folder):
    # .zip fileを目的のfolderに解凍する
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)
        return zip_ref.namelist() 

def rename_and_move_files(extracted_files, year, save_folder):
    # 解凍したfileの名前を変更して移動する
    renamed_files = []
    for file in extracted_files:
        old_path = os.path.join(save_folder, file)
        new_filename = f"{year}_{file}"
        new_path = os.path.join(save_folder, new_filename)
        os.rename(old_path, new_path)
        renamed_files.append(new_path)
    return renamed_files

def process_download_and_extract(url, save_folder, year):
    # .zipのfileをdownloadし、解凍して名前を変更する
    zip_path = download_file(url, save_folder)
    extracted_files = extract_zip(zip_path, save_folder)
    renamed_files = rename_and_move_files(extracted_files, year, save_folder)
    os.remove(zip_path)
    return renamed_files

def process_books(book_dict_list, save_folder):
    # すべてのprocessを統合する
    for book_dict in book_dict_list:
        for book_id, book_info in book_dict.items():
            book_name, book_url = book_info  # 小説の情報を解析する
            
            # ウェブページのHTMLを取得して解析する
            response = requests.get(book_url)
            response.encoding = 'utf-8'
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 初出年份を抽出する
            try:
                initial_publication = soup.find('td', class_='header', string='初出：').find_next('td').get_text(strip=True)
                publication_year = clean_and_extract_publication_year(initial_publication)
            except AttributeError:
                publication_year = "未知"
            
            # download linkを抽出する
            download_links = extract_download_links(soup, os.path.dirname(book_url) + '/')
            
            # fileをdownloadし、解凍して名前を変更する
            for link in download_links:
                renamed_files = process_download_and_extract(link, save_folder, publication_year)
                book_info.extend([publication_year] + renamed_files)

save_folder = '/Users/yandongxin/Downloads/edokawa_text_data'

process_books(books, save_folder)

# 结果を表示する
for book in books:
    print(book)

In [ ]:
def save_books_to_csv(book_dict_list, output_file):
    # list を CSV 形式に変換して保存する
    data = []
    
    for book_dict in book_dict_list:
        for book_id, book_info in book_dict.items():
            book_name = book_info[0]
            book_url = book_info[1]
            publication_year = book_info[2] if len(book_info) > 2 else "未知"
            extracted_files = book_info[3:] if len(book_info) > 3 else []
            extracted_files_str = "; ".join(extracted_files)
            
            data.append([book_id, book_name, book_url, publication_year, extracted_files_str])
    
    df = pd.DataFrame(data, columns=["Book ID", "Book Name", "Download Link", "Publication Year", "File Save Path"])
    df.to_csv(output_file, index=False, encoding="utf-8-sig")

csv_file_path = "/Users/yandongxin/Downloads/edokawa_text_data/books.csv"

save_books_to_csv(books, csv_file_path)

print(f"CSV fileが保存されている場所: {csv_file_path}")


In [1]:


import requests
from bs4 import BeautifulSoup

# 江戸川乱歩の作者ページ URL
url = "https://www.aozora.gr.jp/cards/001779/card56658.html"

# HTTP GET リクエストを送信（ヘッダーなし）
response = requests.get(url)

# エンコーディングを明示的に指定
response.encoding = 'utf-8'

# HTML 全体を取得
html_content = response.text

# HTML 全体を表示
print(html_content)


<html lang="ja" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8">
<meta property="og:type" content="book">
<meta property="og:url" content="https://www.aozora.gr.jp/cards/001779/card56658.html">
<meta property="og:image" content="https://www.aozora.gr.jp/images/top_logo_300x300.png">
<meta property="og:image:type" content="image/png">
<meta property="og:title" content="吸血鬼 (江戸川 乱歩)">
<meta property="og:description" content="作者の言葉　この物語の主人公は、彼（か）のバルカン地方の伝説『吸血鬼』にも比すべき、人界の悪魔である。　一度埋葬された死人が鬼と化して、夜な夜な墓場をさまよい出（い）で、人家に忍び入って、睡眠中の人間…">
<!-- OGP: thanks to @cc4966 https://github.com/aozorahack/ogp -->
<meta name="twitter:card" content="summary" />
<meta http-equiv="Content-Type" content="text/html;charset=utf-8">
<meta http-equiv="Content-Style-Type" content="text/css">
<title>図書カード：吸血鬼</title>
<style type="text/css">
<!--
body{
    margin-left: 10%;
    margin-right: 10%;
    margin-bottom: 5%;
    line-height: 150%;
}
a:hover{
    color: white;
    background-color: #0000ff;
}
h1{
   

In [4]:
import requests
from bs4 import BeautifulSoup
import re

# 目标 URL
url = "https://www.aozora.gr.jp/cards/001779/card57225.html"

# 获取网页内容
response = requests.get(url)
response.encoding = "utf-8"

# 解析 HTML
soup = BeautifulSoup(response.text, "html.parser")

# 获取 "初出：" 所在的 <td>，然后获取它的下一个 <td>
shuppatsu_td = soup.find("td", text="初出：")
text = shuppatsu_td.find_next_sibling("td").get_text() if shuppatsu_td else None

# 提取出版年份的函数
def clean_and_extract_publication_year(text):
    if not text:  # 确保 text 不是 None
        return None

    cleaned_text = re.sub(r"（[^）]*）", "", text)  # 删除括号内的文字
    match = re.search(r"(\d{4})", cleaned_text)  # 查找 4 位年份
    
    return match.group(1) if match else None

# 获取出版年份
publication_year = clean_and_extract_publication_year(text)
print(f"出版年份: {publication_year}")


出版年份: 1960


/var/folders/3r/flw8j3894_q3_v9bv5lc37rw0000gn/T/ipykernel_47140/3676653448.py:16: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  shuppatsu_td = soup.find("td", text="初出：")
